<a href="https://colab.research.google.com/github/madhav037/Learn-AI/blob/main/neural_network_with_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd

In [40]:
class NeuralNetwrok(nn.Module):
  def __init__(self):
    super(NeuralNetwrok, self).__init__()
    self.inputL_hiddenL = nn.Linear(784, 128)
    self.hiddenL_hidden2L = nn.Linear(128, 64)
    self.hidden2L_outputL = nn.Linear(64, 10)

  def forward(self, x):
    x = F.relu(self.inputL_hiddenL(x))
    x = F.relu(self.hiddenL_hidden2L(x))
    x = self.hidden2L_outputL(x)

    return F.softmax(x, dim=1)

In [41]:
model = NeuralNetwrok()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [42]:
epochs = 1000

In [43]:
df = pd.read_csv('/content/sample_data/mnist_train_small.csv')
df.head()

,6,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,...,0.581,0.582,0.583,0.584,0.585,0.586,0.587,0.588,0.589,0.590
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [44]:
X = df.iloc[:, 1:] / 255.0
y_raw = df.iloc[:, 0]

X_tensor = torch.tensor(X.values, dtype=torch.float32)
y_tensor = torch.tensor(y_raw.values, dtype=torch.long)

y = nn.functional.one_hot(y_tensor, num_classes=10)

In [45]:
for epoch in range(epochs):
    model.train()  # set to training mode

    # Forward pass
    outputs = model(X_tensor)
    loss = criterion(outputs, y_tensor)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
      _, predicted = torch.max(outputs, dim=1)
      correct = (predicted == y_tensor).sum().item()
      accuracy = correct / y_tensor.size(0)
      print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}, Accuracy: {accuracy * 100:.2f}%")

Epoch 1/1000, Loss: 2.3027, Accuracy: 7.80%
Epoch 101/1000, Loss: 1.5710, Accuracy: 90.98%
Epoch 201/1000, Loss: 1.5285, Accuracy: 94.37%
Epoch 301/1000, Loss: 1.5109, Accuracy: 95.69%
Epoch 401/1000, Loss: 1.5006, Accuracy: 96.54%
Epoch 501/1000, Loss: 1.4942, Accuracy: 97.01%
Epoch 601/1000, Loss: 1.4903, Accuracy: 97.27%
Epoch 701/1000, Loss: 1.4874, Accuracy: 97.51%
Epoch 801/1000, Loss: 1.4854, Accuracy: 97.65%
Epoch 901/1000, Loss: 1.4839, Accuracy: 97.78%


In [46]:
test_df = pd.read_csv('/content/sample_data/mnist_test.csv')

X_test = test_df.iloc[:, 1:] / 255.0
y_test_raw = test_df.iloc[:, 0]

X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test_raw.values, dtype=torch.long)

model.eval()

with torch.no_grad():
    test_outputs = model(X_test_tensor)

_, test_predicted = torch.max(test_outputs, dim=1)
test_correct = (test_predicted == y_test_tensor).sum().item()
test_accuracy = test_correct / y_test_tensor.size(0)

print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Test Accuracy: 94.73%


In [47]:
def predict_single_image(model, image_tensor, true_label):
    model.eval()

    with torch.no_grad():
        output = model(image_tensor)

    _, predicted_class = torch.max(output, dim=1)

    return predicted_class.item(), true_label

sample_image = X_test_tensor[0].unsqueeze(0)
true_label = y_test_tensor[0].item()

predicted_label, correct_label = predict_single_image(model, sample_image, true_label)

print(f"Predicted label: {predicted_label}, Correct label: {correct_label}")

Predicted label: 2, Correct label: 2
